In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

import joblib


In [3]:
df_testing = pd.read_csv('/content/drive/MyDrive/CattleDiag/Dataset/dataset_testing.csv')
weight = pd.read_csv('/content/drive/MyDrive/CattleDiag/Dataset/symptom_severity.csv')

In [4]:
# Mendapatkan nama kolom dari data frame
cols = df_testing.columns
data = df_testing[cols].values.flatten()

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df_testing.shape)

df_testing = pd.DataFrame(s, columns = df_testing.columns)

In [5]:
# Mengisi data yang kosong dengan nilai 0
df_testing = df_testing.fillna(value=0)

In [6]:
vals = df_testing.values
symptoms = weight['Gejala'].unique()

for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = weight[weight['Gejala'] == symptoms[i]]['Bobot'].values[0]

In [7]:
d = pd.DataFrame(vals, columns=cols)
df_testing['Penyakit'].value_counts()
df_testing['Penyakit'].unique()

data = df_testing.iloc[:,1:].values
labels = df_testing['Penyakit'].values


In [8]:
df_testing.head()

,Penyakit,Gejala 1,Gejala 2,Gejala 3,Gejala 4,Gejala 5,Gejala 6,Gejala 7,Gejala 8,Gejala 9,Gejala 10,Gejala 11,Gejala 12,Gejala 13,Gejala 14,Gejala 15,Gejala 16,Gejala 17,Gejala 18,Gejala 19
0,Pneumonia,15.3,15.3,7.7,2.5,14.2,7.7,2.5,13.0,0,0,0,0,0,0,0,0,0,0,0
1,Diare,9.2,20.3,10.5,7.7,9.2,4.5,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Anthrax,13.0,15.3,15.3,7.7,9.2,9.2,2.5,2.5,0,0,0,0,0,0,0,0,0,0,0
3,Brucellosis,14.2,2.5,2.5,14.2,2.5,2.5,2.5,2.5,0,0,0,0,0,0,0,0,0,0,0
4,Septichaemia Epizootica (SE),15.3,9.2,14.2,2.5,2.5,15.3,2.5,6.2,2.5,2.5,2.5,13.0,0,0,0,0,0,0,0


In [9]:
# Unpickle classifier SVC
svc = joblib.load("/content/drive/MyDrive/CattleDiag/svc.pkl")
pred_svc = svc.predict(data)

print('Label Actual : ', labels)
print('Hasil Prediksi SVC :', pred_svc)
print('SVC F1-score% =', f1_score(labels, pred_svc, average='macro')*100, '|', 'SVC Accuracy% =', accuracy_score(labels, pred_svc)*100)


Label Actual :  ['Pneumonia' 'Diare' 'Anthrax' 'Brucellosis'
 'Septichaemia Epizootica (SE)' 'Malignant Catrrahal Fever'
 'Apthea Epizootica (AE)' 'Bovine Epheral Fever (BEF)' 'Trypanosomiasis'
 'Fasciolasis' 'Nematodosis' 'Thelazia' 'Scabies' 'Bloat' 'Jembrana'
 'Akabane' 'Enzootic Bovine leukosis (EBL)'
 'Infectious Bovine Rhinotracheitis (IBR)' 'Blackleg' 'Colibacillosis'
 'Dermatophilosis' 'Heartwater' 'Orf' 'Leptospirosis' 'Listeriosis'
 'Mastitis' 'Paratuberkulosis' 'Pink eye' 'Salmonellosis' 'Tuberkulosis'
 'Aspergillosis' 'Anaplasmosis' 'Babesiosis']
Hasil Prediksi SVC : ['Pneumonia' 'Diare' 'Anthrax' 'Brucellosis'
 'Septichaemia Epizootica (SE)' 'Malignant Catrrahal Fever'
 'Apthea Epizootica (AE)' 'Bovine Epheral Fever (BEF)' 'Trypanosomiasis'
 'Fasciolasis' 'Nematodosis' 'Thelazia' 'Scabies' 'Bloat' 'Jembrana'
 'Akabane' 'Enzootic Bovine leukosis (EBL)'
 'Infectious Bovine Rhinotracheitis (IBR)' 'Blackleg' 'Colibacillosis'
 'Dermatophilosis' 'Heartwater' 'Orf' 'Leptospirosis

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical  # Importing to_categorical
import joblib

# Assuming you have your data and labels loaded into 'data' and 'labels' variables

# Memuat model yang telah dilatih dan encoder
model = keras.models.load_model('/content/drive/MyDrive/CattleDiag/cattlediag_tf.h5')
encoder = joblib.load("/content/drive/MyDrive/CattleDiag/encoder.pkl")

# Assuming 'labels' variable contains the labels for your data
# Konversi label menggunakan encoder
labels_e = encoder.transform(labels)
labels_c = to_categorical(labels_e, num_classes=41)

# Assuming 'data' variable contains the data for evaluation
# Konversi tipe data ke float32
data = data.astype(np.float32)
labels_c = labels_c.astype(np.float32)

# Konversi array NumPy ke Tensor menggunakan tf.convert_to_tensor
data_tensor = tf.constant(data)
labels_c_tensor = tf.constant(labels_c)

# Evaluasi model dengan tensor
score = model.evaluate(data_tensor, labels_c_tensor, batch_size=32)
pred_tf = model.predict(data_tensor)

actual = np.argmax(labels_c, axis=1)
predicted = np.argmax(pred_tf, axis=1)

print('Label Actual : ', actual)
print(encoder.inverse_transform(actual))

print('Label Predicted : ', predicted)
print(encoder.inverse_transform(predicted))

print('CNN Score : ', score)


2/2 [==============================] - 0s 8ms/step
Label Actual :  [26 12  2  9 29 20  3  8 31 14 22 30 28  7 17  0 13 16  6 10 11 15 23 18
 19 21 24 25 27 32  4  1  5]
['Pneumonia' 'Diare' 'Anthrax' 'Brucellosis'
 'Septichaemia Epizootica (SE)' 'Malignant Catrrahal Fever'
 'Apthea Epizootica (AE)' 'Bovine Epheral Fever (BEF)' 'Trypanosomiasis'
 'Fasciolasis' 'Nematodosis' 'Thelazia' 'Scabies' 'Bloat' 'Jembrana'
 'Akabane' 'Enzootic Bovine leukosis (EBL)'
 'Infectious Bovine Rhinotracheitis (IBR)' 'Blackleg' 'Colibacillosis'
 'Dermatophilosis' 'Heartwater' 'Orf' 'Leptospirosis' 'Listeriosis'
 'Mastitis' 'Paratuberkulosis' 'Pink eye' 'Salmonellosis' 'Tuberkulosis'
 'Aspergillosis' 'Anaplasmosis' 'Babesiosis']
Label Predicted :  [26 12  2  9 29 20  3  8 31 14 22 30 28  7 17  0 13 16  6 10 11 15 23 18
 19 21 24 25 27 32  4  1  5]
['Pneumonia' 'Diare' 'Anthrax' 'Brucellosis'
 'Septichaemia Epizootica (SE)' 'Malignant Catrrahal Fever'
 'Apthea Epizootica (AE)' 'Bovine Epheral Fever (BEF)' '